## Pandas analysis for banking datset

Dataset:
https://www.kaggle.com/datasets/ismetsemedov/transactions?resource=download

In [2]:
#Import libraries
import pandas as pd
import time
import tracemalloc

In [3]:
#Create function to measure and log results
benchmark_results = []

def measure_and_log(step_name, library_name, operation_func, *args, **kwargs):
    import time
    import tracemalloc

    tracemalloc.start()
    start = time.perf_counter()

    result = operation_func(*args, **kwargs)

    end = time.perf_counter()
    _, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    # Build the new entry
    new_entry = {
        "Library": library_name,
        "Step": step_name,
        "Time (s)": round(end - start, 4),
        "Peak Memory (MB)": round(peak / 10**6, 4)
    }

    # Remove existing entry for same library + step
    global benchmark_results
    benchmark_results = [
        entry for entry in benchmark_results
        if not (entry["Library"] == library_name and entry["Step"] == step_name)
    ]

    # Add the new one
    benchmark_results.append(new_entry)

    return result

In [4]:
transactions_csv = '/Users/adolfomytr/Documents/Data Science/Polars/synthetic_fraud_data.csv'

#### Step 1: Load the dataset

In [5]:
def load_csv():
    return pd.read_csv(transactions_csv, parse_dates=["timestamp"])

df = measure_and_log("Load CSV", "Pandas", load_csv)

,transaction_id,customer_id,card_number,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,...,device,channel,device_fingerprint,ip_address,distance_from_home,high_risk_merchant,transaction_hour,weekend_transaction,velocity_last_hour,is_fraud
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820+00:00,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,iOS App,mobile,e8e6160445c935fd0001501e4cbac8bc,197.153.60.199,0,False,0,False,"{'num_transactions': 1197, 'total_amount': 334...",False
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464+00:00,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,Edge,web,a73043a57091e775af37f252b3a32af9,208.123.221.203,1,True,0,False,"{'num_transactions': 509, 'total_amount': 2011...",True
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762+00:00,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,Firefox,web,218864e94ceaa41577d216b149722261,10.194.159.204,0,False,0,False,"{'num_transactions': 332, 'total_amount': 3916...",False
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466+00:00,Gas,major,Exxon,630.60,AUD,Australia,...,iOS App,mobile,70423fa3a1e74d01203cf93b51b9631d,17.230.177.225,0,False,0,False,"{'num_transactions': 764, 'total_amount': 2201...",False
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063+00:00,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,Chrome,web,9880776c7b6038f2af86bd4e18a1b1a4,136.241.219.151,1,False,0,False,"{'num_transactions': 218, 'total_amount': 4827...",True


#### Step 2: Exploratory Data Analysis

In [8]:
summary = measure_and_log("Describe numeric columns", "Pandas", lambda: df.describe())

merchant_type_counts = measure_and_log("Merchang type value counts", "Pandas", lambda: df["merchant_type"].value_counts())

missing_values = measure_and_log("Missing values per column", "Pandas", lambda: df.isnull().sum())

min_date = measure_and_log("Min txn_date", "Pandas", lambda: df["timestamp"].min())
max_date = measure_and_log("Max txn_date", "Pandas", lambda: df["timestamp"].max())

unique_customers = measure_and_log("Unique customer_id count", "Pandas", lambda: df["customer_id"].nunique())


#### Step 3: Transaction Volume Analysis

In [16]:
daily_txns = measure_and_log("Daily transaction count", "Pandas", lambda: df.groupby(df["timestamp"].dt.to_period("D")).size())

monthly_txns = measure_and_log("Monthly transaction count", "Pandas", lambda: df.groupby(df["timestamp"].dt.to_period("M")).size())

/Users/adolfomytr/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1088: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


timestamp
2024-09-30    240937
2024-10-01    241813
2024-10-02    242718
2024-10-03    241204
2024-10-04    241328
2024-10-05    241086
2024-10-06    241880
2024-10-07    241345
2024-10-08    240890
2024-10-09    241514
2024-10-10    241228
2024-10-11    242228
2024-10-12    240633
2024-10-13    241511
2024-10-14    241867
2024-10-15    241458
2024-10-16    241102
2024-10-17    241344
2024-10-18    241286
2024-10-19    239985
2024-10-20    241904
2024-10-21    241638
2024-10-22    241872
2024-10-23    241807
2024-10-24    240947
2024-10-25    241750
2024-10-26    240929
2024-10-27    241735
2024-10-28    241010
2024-10-29    241208
2024-10-30    241609
Freq: D, dtype: int64

#### Step 4: Transaction Amount Aggregation

In [17]:
avg_total_by_merch_type = measure_and_log("Avg/Total txn by merch type", "Pandas", lambda: df.groupby("merchant_type")["amount"].agg(["mean", "sum"]))

top_customers = measure_and_log("Top 10 customers by volume", "Pandas",
    lambda: df.groupby("customer_id")["amount"].sum().sort_values(ascending=False).head(10)
)


In [18]:
benchmark_table = pd.DataFrame(benchmark_results)
benchmark_table

,Library,Step,Time (s),Peak Memory (MB)
0,Pandas,Load CSV,150.6353,6036.7737
1,Pandas,Describe numeric columns,0.9219,368.1902
2,Pandas,Merchang type value counts,0.7321,14.9709
3,Pandas,Missing values per column,8.3195,538.8557
4,Pandas,Min txn_date,0.0574,67.4244
5,Pandas,Max txn_date,0.0142,14.9689
6,Pandas,Unique customer_id count,0.3801,59.9378
7,Pandas,Daily transaction count,0.3121,187.1015
8,Pandas,Monthly transaction count,0.2372,187.0982
9,Pandas,Avg/Total txn by merch type,0.7385,119.7579
